In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import (
    Layer, Input, Dense, Dropout, LayerNormalization,
    MultiHeadAttention, Add, GlobalAveragePooling2D,
    Lambda
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, precision_score
from tensorflow.keras.metrics import Precision, Recall, AUC

import matplotlib.pyplot as plt

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Load Data
def load_data_with_generator(dataset_path, image_size, batch_size, limit_per_class):
    datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.resnet.preprocess_input, 
        horizontal_flip=True,
        vertical_flip=True,
        rotation_range=120,
        width_shift_range=0.3,
        height_shift_range=0.3,
        zoom_range=0.3,
        shear_range=0.3,
        brightness_range=[0.1, 1.5],
        validation_split=0.2  # Use this for splitting training and validation
    )

    train_generator = datagen.flow_from_directory(
        dataset_path,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = datagen.flow_from_directory(
        dataset_path,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    return train_generator, validation_generator

# Class Token
class ClassToken(Layer):
    def __init__(self, **kwargs):
        super(ClassToken, self).__init__(**kwargs)

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.class_token = tf.Variable(
            initial_value=w_init(shape=(1, 1, input_shape[-1]), dtype="float16"),
            trainable=True,
            name="class_token"
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        broadcasted_class_token = tf.tile(self.class_token, [batch_size, 1, 1])
        return tf.keras.layers.concatenate([broadcasted_class_token, inputs], axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1] + 1, input_shape[2])

# Transformer Encoder
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.2):
    x = LayerNormalization(epsilon=1e-6)(inputs)
    attention_out = MultiHeadAttention(num_heads=num_heads, key_dim=head_size, dropout=dropout)(x, x)
    x = Add()([attention_out, inputs])
    x = LayerNormalization(epsilon=1e-6)(x)
    ff_out = Dense(ff_dim, activation="relu", kernel_regularizer=l2(1e-4))(x)
    ff_out = Dropout(dropout)(ff_out)
    ff_out = Dense(inputs.shape[-1], kernel_regularizer=l2(1e-4))(ff_out)
    ff_out = Dropout(dropout)(ff_out)
    x = Add()([ff_out, x])
    return x

# ResNet50 Transformer Model
def ResNet50Transformer(input_shape, num_classes, transformer_layers, head_size, num_heads, ff_dim, dropout, fine_tune_at=50):
    inputs = Input(shape=input_shape)

    resnet_model = ResNet50(include_top=False, weights='imagenet', input_tensor=inputs)

    if fine_tune_at is not None:
        for layer in resnet_model.layers[:fine_tune_at]:
            if not isinstance(layer, tf.keras.layers.BatchNormalization):
                layer.trainable = False

    x = resnet_model.output
    x = GlobalAveragePooling2D()(x)
    x = tf.expand_dims(x, axis=1)
    x = ClassToken()(x)
    for _ in range(transformer_layers):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = Dense(4096, activation="gelu")(x)
    x = Dropout(0.2)(x)

    x = Lambda(lambda x: x[:, 0])(x)
    x = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=x)
    return model

def train_model(model, train_generator, config, num_epochs=120):
    print(f"Total number of training samples: {len(train_generator)}")

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        for batch_start in range(0, len(train_generator), config["batch_size"] * config["acc_steps"]):
            batch_end = min(batch_start + config["batch_size"] * config["acc_steps"], len(train_generator))
            print(f"Batch indices: {batch_start} to {batch_end}")

            batch_images, batch_labels = train_generator.next()

            with tf.GradientTape() as tape:
                predictions = model(batch_images)
                loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(batch_labels, predictions))

            gradients = tape.gradient(loss, model.trainable_variables)
            model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))

            print(f"Batch Loss: {loss}")

        model.reset_metrics()
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.numpy()}")

def evaluate_model(model, test_images, test_labels, config):
    test_loss = 0.0
    test_accuracy = 0.0
    num_batches = len(test_images) // config["batch_size"]

    for i in range(num_batches):
        start_idx = i * config["batch_size"]
        end_idx = (i + 1) * config["batch_size"]

        batch_images = np.array(test_images[start_idx:end_idx])
        batch_labels = np.array(test_labels[start_idx:end_idx])

        batch_loss, batch_accuracy, _, _ = model.evaluate(batch_images, batch_labels, batch_size=config["batch_size"])
        test_loss += batch_loss
        test_accuracy += batch_accuracy

    average_test_loss = test_loss / num_batches
    average_test_accuracy = test_accuracy / num_batches

    print(f"Average Test Loss: {average_test_loss}, Average Test Accuracy: {average_test_accuracy}")




if __name__ == "__main__":
    config = {
        "num_layers": 6,
        "hidden_dim": 3072,
        "mlp_dim": 4096,
        "num_heads": 32,
       # "patch_size":64,
        "dropout_rate": 0.2,
        "image_size": 150,
        "num_channels": 3,
        "num_classes": 8,
        "batch_size":32,
        "acc_steps": 4,
        "epochs": 120
    }
   # config["num_patches"] = (config["image_size"] // config["patch_size"]) ** 2
    limit_per_class = None
    train_generator, val_generator = load_data_with_generator(
        '//kaggle/input/kather5000images/Kather_texture_2016_image_tiles_3500', config["image_size"], config["batch_size"], limit_per_class
    )

    model = ResNet50Transformer(
        input_shape=(150, 150, 3),
        num_classes=config["num_classes"],
        transformer_layers=config["num_layers"],
        head_size=128,
        num_heads=config["num_heads"],
        ff_dim=4096,
        dropout=config["dropout_rate"]
    )

    initial_learning_rate = 0.0001

    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate, decay_steps=10000, alpha=0.00001
    )
    adam_optimizer = Adam(learning_rate=lr_schedule)

    optimizer = tf.keras.mixed_precision.LossScaleOptimizer(
        adam_optimizer,
        dynamic=True
    )

    
        
    model.compile(
    optimizer=optimizer, 
    loss='categorical_crossentropy', 
    metrics=['accuracy', Precision(), Recall()]
    )
    checkpoint = ModelCheckpoint('best_model_kather_adamV1.h5', save_best_only=True)

    csv_logger = CSVLogger('training_katherv1.log')

    history = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=config["epochs"],
        validation_data=val_generator,
        callbacks=[checkpoint, csv_logger],
        verbose=1
    )

    model.save('trained_model_kather_adamV1.h5')


plt.figure(figsize=(18, 6))


plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()


plt.tight_layout()
plt.show()

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet import preprocess_input
from tifffile import imread
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

class ClassToken(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(ClassToken, self).__init__(**kwargs)

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.class_token = tf.Variable(
            initial_value=w_init(shape=(1, 1, input_shape[-1]), dtype="float16"),
            trainable=True,
            name="class_token"
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        broadcasted_class_token = tf.tile(self.class_token, [batch_size, 1, 1])
        return tf.keras.layers.concatenate([broadcasted_class_token, inputs], axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1] + 1, input_shape[2])

model = load_model('/kaggle/input/trainedkathrer/best_model_kather_adamV1.h5', custom_objects={'ClassToken': ClassToken})

class_names = ['01_TUMOR', '02_STROMA', '03_COMPLEX', '04_LYMPHO', '05_DEBRIS', '06_MUCOSA', '07_ADIPOSE', '08_EMPTY']

test_images_folder = '/kaggle/input/kather5000images/Kather_texture_2016_image_tiles_1500'

total_predictions = 0
correct_predictions = 0
all_true_labels = []
all_predicted_labels = []

# Initialize TP, FP, TN, FN counters
TP = 0
FP = 0
TN = 0
FN = 0

print("Wrong identified images")
for class_folder in class_names:
    class_folder_path = os.path.join(test_images_folder, class_folder)

    for filename in os.listdir(class_folder_path):
        if filename.endswith(".tif"):
            image_path = os.path.join(class_folder_path, filename)
            img = imread(image_path)
            img_array = img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)

            predictions = model.predict(img_array, verbose=0)

            predicted_class_index = np.argmax(predictions)
            predicted_class_name = class_names[predicted_class_index]

            all_true_labels.append(class_folder)
            all_predicted_labels.append(predicted_class_name)

            # Check if actual class matches predicted class
            if predicted_class_name != class_folder:
                # Print only when the actual class doesn't match the predicted class
                print(f"Image: {filename}, Actual Class: {class_folder}, Predicted Class: {predicted_class_name}")

            # Calculate TP, FP, TN, FN
            if predicted_class_name == class_folder:
                correct_predictions += 1
                TP += 1
            else:
                FP += 1
                FN += 1

            total_predictions += 1

accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print TP, FP, TN, FN
print("True Positive (TP):", TP)
print("False Positive (FP):", FP)
print("True Negative (TN):", TN)
print("False Negative (FN):", FN)

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels, labels=class_names)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

plt.xticks(np.arange(len(class_names)), class_names, rotation=45)
plt.yticks(np.arange(len(class_names)), class_names)

plt.show()

classification_rep = classification_report(all_true_labels, all_predicted_labels, target_names=class_names)
print("Classification Report:\n", classification_rep)

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet import preprocess_input
from tifffile import imread
from sklearn.metrics import (
    confusion_matrix, classification_report,
    precision_score, recall_score, f1_score, hamming_loss,
    cohen_kappa_score, matthews_corrcoef
)
from scipy.stats import pearsonr
import seaborn as sns
import matplotlib.pyplot as plt

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

class ClassToken(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(ClassToken, self).__init__(**kwargs)

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.class_token = tf.Variable(
            initial_value=w_init(shape=(1, 1, input_shape[-1]), dtype="float16"),
            trainable=True,
            name="class_token"
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        broadcasted_class_token = tf.tile(self.class_token, [batch_size, 1, 1])
        return tf.keras.layers.concatenate([broadcasted_class_token, inputs], axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1] + 1, input_shape[2])

model = load_model('/kaggle/input/trainedkathrer/best_model_kather_adamV1.h5', custom_objects={'ClassToken': ClassToken})

class_names = ['01_TUMOR', '02_STROMA', '03_COMPLEX', '04_LYMPHO', '05_DEBRIS', '06_MUCOSA', '07_ADIPOSE', '08_EMPTY']

test_images_folder = '/kaggle/input/kather5000images/Kather_texture_2016_image_tiles_1500'

all_true_labels = []
all_predicted_labels = []
all_predicted_probs = []  # New list to store probabilities

print("Processing images...")
for class_folder in class_names:
    class_folder_path = os.path.join(test_images_folder, class_folder)

    for filename in os.listdir(class_folder_path):
        if filename.endswith(".tif"):
            image_path = os.path.join(class_folder_path, filename)
            img = imread(image_path)
            img_array = img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)

            predictions = model.predict(img_array, verbose=0)
            predicted_class_index = np.argmax(predictions)
            predicted_class_name = class_names[predicted_class_index]

            all_true_labels.append(class_folder)
            all_predicted_labels.append(predicted_class_index)
            all_predicted_probs.append(predictions[0])  # Store probabilities


# Convert class names to numbers for metrics calculation
true_labels_numeric = [class_names.index(label) for label in all_true_labels]


# Calculating various metrics
precision = precision_score(true_labels_numeric, all_predicted_labels, average='weighted')
recall = recall_score(true_labels_numeric, all_predicted_labels, average='weighted')  # Sensitivity
f1 = f1_score(true_labels_numeric, all_predicted_labels, average='weighted')
ham_loss = hamming_loss(true_labels_numeric, all_predicted_labels)
kappa = cohen_kappa_score(true_labels_numeric, all_predicted_labels)
mcc = matthews_corrcoef(true_labels_numeric, all_predicted_labels)

# Confusion Matrix
conf_matrix = confusion_matrix(true_labels_numeric, all_predicted_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.show()

# Classification Report
classification_rep = classification_report(true_labels_numeric, all_predicted_labels, target_names=class_names)
print("Classification Report:\n", classification_rep)

# Printing the metrics
print(f"Precision: {precision:.2f}")
print(f"Recall/Sensitivity: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Hamming Loss: {ham_loss:.2f}")
print(f"Cohen's Kappa Score: {kappa:.2f}")
print(f"Matthews Correlation Coefficient: {mcc:.2f}")

# Calculate the Pearson correlation coefficient
predicted_probs_true_class = [probs[class_index] for probs, class_index in zip(all_predicted_probs, true_labels_numeric)]
correlation, _ = pearsonr(predicted_probs_true_class, true_labels_numeric)
print(f"Pearson Correlation Coefficient: {correlation:.2f}")

# Calculating and printing class-wise accuracy
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
print("Class-wise Accuracy:")
for class_name, accuracy in zip(class_names, class_accuracies):
    print(f"{class_name}: {accuracy:.2f}")


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet import preprocess_input
from tifffile import imread
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

class ClassToken(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(ClassToken, self).__init__(**kwargs)

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.class_token = tf.Variable(
            initial_value=w_init(shape=(1, 1, input_shape[-1]), dtype="float16"),
            trainable=True,
            name="class_token"
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        broadcasted_class_token = tf.tile(self.class_token, [batch_size, 1, 1])
        return tf.keras.layers.concatenate([broadcasted_class_token, inputs], axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1] + 1, input_shape[2])

# Load the model
model = load_model('/kaggle/input/trainedkathrer/best_model_kather_adamV1.h5', custom_objects={'ClassToken': ClassToken})

class_names = ['01_TUMOR', '02_STROMA', '03_COMPLEX', '04_LYMPHO', '05_DEBRIS', '06_MUCOSA', '07_ADIPOSE', '08_EMPTY']
test_images_folder = '/kaggle/input/kather5000images/Kather_texture_2016_image_tiles_1500'

# Modify the model to create a feature extractor
feature_extractor_layer_index = -3  # Assuming the third last layer is the desired feature extractor layer
feature_extractor = tf.keras.models.Model(
    inputs=model.inputs, 
    outputs=model.layers[feature_extractor_layer_index].output
)

# Initialize lists for features and labels
features = []
labels = []

# Extract features from the test dataset
for class_folder in class_names:
    class_folder_path = os.path.join(test_images_folder, class_folder)
    for filename in os.listdir(class_folder_path):
        if filename.endswith(".tif"):
            image_path = os.path.join(class_folder_path, filename)
            img = imread(image_path)
            img_array = img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)

            feature = feature_extractor.predict(img_array)
            features.append(feature[0])
            labels.append(class_folder)

# Convert to numpy arrays
# Convert to numpy arrays and flatten the features
features = np.array(features)
features = features.reshape(features.shape[0], -1)  # Flatten each feature vector
labels = np.array(labels)

# Apply t-SNE for dimensionality reduction
tsne = TSNE(n_components=2, random_state=0)
reduced_features = tsne.fit_transform(features)

# Plotting
plt.figure(figsize=(12, 8))
for i, class_name in enumerate(class_names):
    indices = np.where(labels == class_name)
    plt.scatter(reduced_features[indices, 0], reduced_features[indices, 1], label=class_name)

plt.title('t-SNE of extracted features')
plt.xlabel('t-SNE feature 1')
plt.ylabel('t-SNE feature 2')
plt.legend()
plt.show()


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet import preprocess_input
from tifffile import imread
import matplotlib.pyplot as plt
from tensorflow.keras import Model
import matplotlib.cm as cm
import matplotlib

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# Custom ClassToken Layer
class ClassToken(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(ClassToken, self).__init__(**kwargs)

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.class_token = tf.Variable(
            initial_value=w_init(shape=(1, 1, input_shape[-1]), dtype="float16"),
            trainable=True,
            name="class_token"
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        broadcasted_class_token = tf.tile(self.class_token, [batch_size, 1, 1])
        return tf.keras.layers.concatenate([broadcasted_class_token, inputs], axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1] + 1, input_shape[2])

# Load the trained model
model = load_model('/kaggle/input/trainedkathrer/best_model_kather_adamV1.h5', custom_objects={'ClassToken': ClassToken})
class_names = ['01_TUMOR', '02_STROMA', '03_COMPLEX', '04_LYMPHO', '05_DEBRIS', '06_MUCOSA', '07_ADIPOSE', '08_EMPTY']
test_images_folder = '/kaggle/input/kather5000images/Kather_texture_2016_image_tiles_1500'

# Grad-CAM Function
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        loss = predictions[:, pred_index]

    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))
    heatmap = tf.matmul(output, pooled_grads[..., tf.newaxis])
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def display_gradcam(img, heatmap, alpha=0.5):
    img = tf.keras.preprocessing.image.img_to_array(img)
    
    # Handle NaN or Inf values in heatmap
    heatmap = np.nan_to_num(heatmap)

    heatmap = np.uint8(255 * heatmap)

    # Use the recommended method for colormap
    jet = matplotlib.cm.get_cmap("jet")

    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)

    return superimposed_img

# Class names and Test Images Folder

# Visualization for First Five Images from Each Class
def get_top_images_with_confidences(class_folder_path, model, num_images=20):
    image_predictions = []
    for filename in os.listdir(class_folder_path):
        if filename.endswith(".tif"):
            image_path = os.path.join(class_folder_path, filename)
            img = imread(image_path)
            img_array = img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
            
            predictions = model.predict(img_array,verbose=0)
            image_predictions.append((filename, predictions[0]))  # Store filename and prediction probabilities

    # Sort the images by max confidence and get the top 'num_images'
    top_images = sorted(image_predictions, key=lambda x: np.max(x[1]), reverse=True)[:num_images]
    return top_images

# Visualization for Top Images from Each Class
for class_folder in class_names:
    class_folder_path = os.path.join(test_images_folder, class_folder)
    top_images_with_confidences = get_top_images_with_confidences(class_folder_path, model, num_images=5)

    for filename, confidences in top_images_with_confidences:
        image_path = os.path.join(class_folder_path, filename)
        img = imread(image_path)
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        # Generate Grad-CAM heatmap
        heatmap = make_gradcam_heatmap(img_array, model, 'conv5_block3_3_conv')  # Adjust the layer name as needed
        superimposed_img = display_gradcam(img, heatmap)

        # Displaying the original image, Grad-CAM image, and confidence levels
        plt.figure(figsize=(12, 4))
        
        plt.subplot(1, 3, 1)
        plt.imshow(img)
        plt.title("Original")
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(superimposed_img)
        plt.title(f"Grad-CAM - {class_folder}")
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.bar(class_names, confidences)
        plt.title("Class Confidence Levels")
        plt.xticks(rotation=45)
        plt.ylabel("Confidence")
        plt.tight_layout()

        plt.show()


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet import preprocess_input
from tifffile import imread
import matplotlib.pyplot as plt
from tensorflow.keras import Model
import matplotlib.cm as cm
import matplotlib
import random

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# Custom ClassToken Layer
class ClassToken(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(ClassToken, self).__init__(**kwargs)

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.class_token = tf.Variable(
            initial_value=w_init(shape=(1, 1, input_shape[-1]), dtype="float16"),
            trainable=True,
            name="class_token"
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        broadcasted_class_token = tf.tile(self.class_token, [batch_size, 1, 1])
        return tf.keras.layers.concatenate([broadcasted_class_token, inputs], axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1] + 1, input_shape[2])

# Load the trained model
model = load_model('/kaggle/input/trainedkathrer/best_model_kather_adamV1.h5', custom_objects={'ClassToken': ClassToken})
class_names = ['01_TUMOR', '02_STROMA', '03_COMPLEX', '04_LYMPHO', '05_DEBRIS', '06_MUCOSA', '07_ADIPOSE', '08_EMPTY']
test_images_folder = '/kaggle/input/kather5000images/Kather_texture_2016_image_tiles_1500'

# Grad-CAM Function
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        loss = predictions[:, pred_index]

    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))
    heatmap = tf.matmul(output, pooled_grads[..., tf.newaxis])
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def display_gradcam(img, heatmap, alpha=0.5):
    img = tf.keras.preprocessing.image.img_to_array(img)
    
    # Handle NaN or Inf values in heatmap
    heatmap = np.nan_to_num(heatmap)

    heatmap = np.uint8(255 * heatmap)

    # Use the recommended method for colormap
    jet = matplotlib.cm.get_cmap("jet")

    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)

    return superimposed_img

# Class names and Test Images Folder

# Visualization for First Five Images from Each Class
def get_random_images(class_folder_path, model, num_images=15):
    all_filenames = [f for f in os.listdir(class_folder_path) if f.endswith(".tif")]
    selected_filenames = random.sample(all_filenames, min(num_images, len(all_filenames)))
    
    image_predictions = []
    for filename in selected_filenames:
        image_path = os.path.join(class_folder_path, filename)
        img = imread(image_path)
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        
        predictions = model.predict(img_array, verbose=0)
        image_predictions.append((filename, predictions[0]))  # Store filename and prediction probabilities

    return image_predictions

for class_folder in class_names:
    class_folder_path = os.path.join(test_images_folder, class_folder)
    random_images_with_confidences = get_random_images(class_folder_path, model, num_images=15)

    for filename, confidences in random_images_with_confidences:
        image_path = os.path.join(class_folder_path, filename)
        img = imread(image_path)
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        heatmap = make_gradcam_heatmap(img_array, model, 'conv5_block3_3_conv')
        superimposed_img = display_gradcam(img, heatmap)

        plt.figure(figsize=(8, 4))

        # Display the Grad-CAM image
        plt.subplot(1, 2, 1)
        plt.imshow(superimposed_img)
        plt.title(f"Grad-CAM - {class_folder}")
        plt.axis('off')

        # Display the confidence percentages
        plt.subplot(1, 2, 2)
        confidence_text = "\n".join([f"{class_name}: {100 * conf:.2f}%" for class_name, conf in zip(class_names, confidences)])
        plt.text(0, 0.5, confidence_text, ha='left', va='center', transform=plt.gca().transAxes)
        plt.axis('off')

        plt.tight_layout()
        plt.show()